# Mass spectrometry data

The objective of this exercise is to read in raw peptide MSMS spectrum information and output a dataframe.
The .msp file can be downloaded [here](https://chemdata.nist.gov/download/peptide_library/libraries/cptaclib/2015/cptac2_mouse_hcd_selected.msp.tar.gz).

The information in this ASCII based text file is organized spectrum by spectrum.
The first line per spectrum provides formatted like this:

&emsp;<code>Name: sequence/charge_nmods_collisionenergy</code>

followed by a comment section which can be disregarded and the actual spectrum data which is tab-separated:

&emsp;<code>m/z&emsp;intensity&emsp;additional_info</code>

Spectra are separated by an empty line.

Code a function that returns two DataFrames or arrays containing the processed and filtered data. The first one should contain the spectrum information (n_spectra, n_m/z_features) and the second one the sequences per row (n_spectra).

Here are some general guidelines:

* The m/z values need to be binned to integer values (mathematically rounded), otherwise the dataframe size would get out of hand. This will allow for multiple values mapped to a single bin (e.g. if there are peaks at 145.1 and 145.2). Here, only the maximum of those peaks should be kept in the final dataframe.

* Rows that are all-zero should be dropped.

Your function should allow for selecting a range on the x-axis (m/z-range). All peaks outside this range can be disregarded. Furthermore, only spectra within a set collision energy range and a maximum sequence length should be contained in the output dataframe.

The faster your function runs, the better. I will time them all in the end.

In [335]:
import numpy as np
import pandas as pd
import timeit

In [336]:
def to_beconsidered(line, sequence, min_ce, max_ce, max_seq_len):
    conditions = True
    ce = float(line.split("_")[-1][:-2])
    if ce < min_ce or ce > max_ce:
        conditions = False
    elif len(sequence) > max_seq_len:
        conditions = False
    return conditions

def get_description_parameters(line, min_ce, max_ce, max_seqlen):
    name = str(line[6:])
    sequence = line.split("/")[0][6:]
    condition = to_beconsidered(line, sequence, min_ce, max_ce, max_seqlen)
    return name, sequence, condition


In [337]:
def msp_to_df(
    input_file,
    max_seq_len=30,
    min_ce=36,
    max_ce=40, # only use these entries, hat eV als Einheit
    mz_min=135, # only record these if entry is used
    mz_max=1400,
):
    """
    Function to read spectrum data from .msp file and convert to dataframe.
    Args:
        input_file (str): path to .msp file
        max_seq_len (int): maximum acceptable sequence length
        min_ce (int): minimum collision energy of spectra to be included in df
        max_ce (int): maximum collision energy of spectra to be included in df
        mz_min (int): lower boundary for m/z to be included in df
        mz_max (int): lower boundary for m/z to be included in df

    Returns:
        df (pd.DataFrame or np.array): spectrum information within defined parameters [n_spectra, n_features]
        seqs (pd.DataFrame or np.array): sequences
    """
    col = ["Name", "Sequence"] + list(np.arange(mz_min,mz_max+1,1))
    ms_df = pd.DataFrame(columns = col)

    counter = 0

    for line in input_file:
        line = line.strip()
        if line[:4] == "Name":
            name, sequence, condition = get_description_parameters(line, min_ce, max_ce, max_seq_len)
            if condition == True:
                ms_df = ms_df.append(pd.Series(0, index=ms_df.columns), ignore_index=True)
                ms_df.at[counter, "Name"] = name
                ms_df.at[counter, "Sequence"] = sequence
                counter += 1

        if condition == True:
            if line[0:1] in ["0","1","2","3","4","5","6","7","8","9"]:
                m_z = round(float(line.split("\t")[0]),0)
                intensity = float(line.split("\t")[1])
                if mz_min <= m_z <= mz_max:
                    if ms_df.at[counter - 1, m_z] < intensity:
                        ms_df.at[counter - 1, m_z] = intensity

        #if counter == 10:
        #    break

    ms_df = ms_df.loc[~(ms_df.iloc[:, 2:]==0).all(axis=1)]
    return ms_df

In [338]:
with open ("../../data/cptac2_mouse_hcd_selected.msp", 'r') as in_file:
    ms_df = msp_to_df(in_file)

In [339]:
ms_df

,Name,Sequence,135,136,137,138,139,140,141,142,...,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400
0,AAAFEDQENETVVVK/2_0_38.7eV,AAAFEDQENETVVVK,0,2851.7,0,0,0,0,1404,0,...,0,0,0,0,0,0,0,0,2797.6,0
1,AAGTIQTSVQEVNSK/2_0_37.3eV,AAGTIQTSVQEVNSK,0,2626,0,0,1553.4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AAGVSVEPFWPGLFAK/2_0_39.3eV,AAGVSVEPFWPGLFAK,0,59857.3,8295.8,0,3220.5,0,24351.6,2054,...,0,0,0,0,0,0,0,0,0,0
3,"AAHSQCAYSNPEGTVLLACEESR/3_2(5,C,CAM)(18,C,CAM)...",AAHSQCAYSNPEGTVLLACEESR,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"AAHSQCAYSNPEGTVLLACEESR/3_2(5,C,CAM)(18,C,CAM)...",AAHSQCAYSNPEGTVLLACEESR,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1946,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2128,YVSHFETDGPHVLLYFDSVPTTR/3_0_39.5eV,YVSHFETDGPHVLLYFDSVPTTR,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2129,YVTLIYTNYENGK/2_0_38.4eV,YVTLIYTNYENGK,0,51469.1,2589.3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2130,YYTYLVMNK/2_0_37eV,YYTYLVMNK,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2131,"YYTYLVMNK/2_1(6,M,Oxidation)_38eV",YYTYLVMNK,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
